In [134]:
import opensim as osim

In [135]:
import numpy as np
import math as m

import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd

### Import Transformed Markers

In [136]:
markers_df = pd.read_csv('model_update/tps_warping_results/markers_transformed.csv', index_col=0)
markers_df.head()

,body,location
name,,
ASIS_l,pelvis,[ 1.91379638e-09 1.91379640e-09 -1.08574441e-01]
ASIS_r,pelvis,[-1.91379637e-09 -1.91379640e-09 1.08574441e-01]
PSIS_l,pelvis,[-0.12953896 0.01857114 -0.0453012 ]
PSIS_r,pelvis,[-0.12796796 0.01822429 0.04270595]
pub_infer_c,pelvis,[-0.03031937 -0.10349564 0.00052657]


In [137]:
def return_dataframe(markers_df, body):
    extract_df = markers_df[markers_df['body']== body]['location']
    ind = extract_df.index
    col = ['x', 'y', 'z']
    list_of_data = [row[1:-1].split() for row in extract_df]
    #print(list_of_data)
    convert = lambda x: 0 if x == '.' else float(x)
    data = np.array([[convert(x) for x in row] for row in list_of_data])
    data_df = pd.DataFrame(data, index=ind, columns=col)
    return data_df


In [138]:
pelvis_df = return_dataframe(markers_df, 'pelvis')
femur_r_df =  return_dataframe(markers_df, 'femur_r')
femur_l_df =  return_dataframe(markers_df, 'femur_l')
tibia_r_df =  return_dataframe(markers_df, 'tibia_r')
tibia_l_df =  return_dataframe(markers_df, 'tibia_l')
patella_r_df =  return_dataframe(markers_df, 'patella_r')
patella_l_df =  return_dataframe(markers_df, 'patella_l')
pelvis_df.tail(10)

,x,y,z
name,,,
ilium_l,-0.065411,0.055832,-0.108336
isch_spine_r,-0.096107,-0.071231,0.051982
isch_spine_l,-0.096069,-0.068601,-0.051277
isch_tuber_r,-0.100033,-0.106089,0.059559
isch_tuber_l,-0.097093,-0.100014,-0.066277
isch_infer_r,-0.059252,-0.126569,0.037440
isch_infer_l,-0.056334,-0.125948,-0.041044
femur_r_center_in_pelvis,-0.049354,-0.071971,0.086060
femur_l_center_in_pelvis,-0.045814,-0.069148,-0.086186


### Parse OSIM scaled model

In [139]:
tree=ET.parse("scaled_model_P03.osim")
root = tree.getroot()
# temp = mean_skin_markers_df*mean_skin_markers_df

#### Location of skin markers in child body frames

In [140]:
imported_markers_dict = {}
for marker in root.iter('Marker'):
    name = marker.attrib['name']
    body = marker.find('socket_parent_frame').text[9:]
    location = [float(val) for val in (marker.find('location').text).split()]
    imported_markers_dict[name] = [body] + location
    
imported_markers_df = pd.DataFrame.from_dict(imported_markers_dict).T
imported_markers_df = imported_markers_df.rename(columns={0:'body', 1:'x', 2:'y', 3:'z'})

In [141]:
imported_markers_df.index

Index(['C7', 'RBAK', 'CLAV', 'STRN', 'T10', 'RASI', 'RPSI', 'LPSI', 'LASI',
       'PE01', 'PE02', 'PE03', 'RTH1', 'RTH2', 'RTH3', 'RGT', 'RKNE', 'RMKNE',
       'LTH1', 'LTH2', 'LTH3', 'LGT', 'LKNE', 'LMKNE', 'RTB1', 'RTB2', 'RTB3',
       'RANK', 'RMMA', 'LTB1', 'LTB2', 'LTB3', 'LANK', 'LMMA', 'RHEE', 'RD5M',
       'RTOE', 'LHEE', 'LD5M', 'LTOE', 'torso_origin_in_pelvis',
       'pelvis_origin_in_ground', 'femur_l_center_in_pelvis',
       'femur_r_center_in_pelvis', 'knee_l_center_in_femur_l',
       'knee_r_center_in_femur_r', 'patella_l_in_femur_l',
       'patella_r_in_femur_r', 'ankle_l_tibia_l_offset',
       'ankle_r_tibia_r_offset'],
      dtype='object')

In [142]:
mrkrs_torso = imported_markers_df[imported_markers_df['body'] == 'torso'][['x', 'y', 'z']]
mrkrs_pelvis = imported_markers_df[imported_markers_df['body'] == 'pelvis'][['x', 'y', 'z']]
mrkrs_femur_r = imported_markers_df[imported_markers_df['body'] == 'femur_r'][['x', 'y', 'z']]
mrkrs_femur_l = imported_markers_df[imported_markers_df['body'] == 'femur_l'][['x', 'y', 'z']]
mrkrs_tibia_r = imported_markers_df[imported_markers_df['body'] == 'tibia_r'][['x', 'y', 'z']]
mrkrs_tibia_l = imported_markers_df[imported_markers_df['body'] == 'tibia_l'][['x', 'y', 'z']]
mrkrs_foot_r = imported_markers_df.loc[['RHEE', 'RD5M','RTOE'], ['x', 'y', 'z']]
mrkrs_foot_l = imported_markers_df.loc[['LHEE', 'LD5M', 'LTOE'], ['x', 'y', 'z']]
                                         

In [143]:
mrkrs_femur_r

,x,y,z
RTH1,-0.016029,-0.21608,0.096395
RTH2,-0.000393,-0.289999,0.071181
RTH3,0.026236,-0.234511,0.077276
RGT,0.001705,0.022386,0.111934
RKNE,-0.000538,-0.367112,0.055837
RMKNE,-0.012286,-0.370174,-0.057258
knee_r_center_in_femur_r,0.000293,-0.379892,-0.001135
patella_r_in_femur_r,0.041636,-0.39349,0.0


In [144]:
import plotly.express as px
import plotly.graph_objects as go
fig = px.scatter(width = 800, height = 600)
fig.add_trace(go.Scatter3d(x = pelvis_df['x'], y = pelvis_df['y'], z=pelvis_df['z'], mode = 'markers', name = 'pelvis_df'))
fig.add_trace(go.Scatter3d(x = mrkrs_pelvis['x'],  
                           y = mrkrs_pelvis['y'], 
                           z = mrkrs_pelvis['z'], mode = 'markers', name = 'osim skin mrkrs'))
fig.update_scenes(aspectmode="data" )
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=2, y=2, z=2)
)
fig.update_layout(scene_camera=camera)
fig.show()

Skin markers and transformed muscles and markers fall essentially into the same set of axes. However, PE markers seem to indicate a significant lordosis...

### Translation of joint centres

In [145]:
# transfomed markers
femur_r_center_in_pelvis = pelvis_df.loc['femur_r_center_in_pelvis']
femur_l_center_in_pelvis = pelvis_df.loc['femur_l_center_in_pelvis']

knee_l_center_in_femur = femur_l_df.loc['knee_l_center_in_femur_l']
knee_r_center_in_femur = femur_r_df.loc['knee_r_center_in_femur_r']

patella_l_center_in_femur = femur_l_df.loc['patella_l_in_femur_l']
patella_r_center_in_femur = femur_r_df.loc['patella_r_in_femur_r']

tibia_l_center_in_tibia = tibia_l_df.loc['tibia_l_center']
tibia_r_center_in_tibia = tibia_r_df.loc['tibia_r_center']

talus_l_center_in_tibia = tibia_l_df.loc['talus_l_center_in_tibia']
talus_r_center_in_tibia = tibia_r_df.loc['talus_r_center_in_tibia']

torso_origin_in_pelvis = pelvis_df.loc['torso_origin_in_pelvis']

In [146]:
mrkrs_femur_r

,x,y,z
RTH1,-0.016029,-0.21608,0.096395
RTH2,-0.000393,-0.289999,0.071181
RTH3,0.026236,-0.234511,0.077276
RGT,0.001705,0.022386,0.111934
RKNE,-0.000538,-0.367112,0.055837
RMKNE,-0.012286,-0.370174,-0.057258
knee_r_center_in_femur_r,0.000293,-0.379892,-0.001135
patella_r_in_femur_r,0.041636,-0.39349,0.0


In [147]:
# markers in the current OSIM scaled model
old_femur_r_in_pelvis = mrkrs_pelvis.loc['femur_r_center_in_pelvis']
old_femur_l_in_pelvis = mrkrs_pelvis.loc['femur_l_center_in_pelvis']

old_knee_r_in_femur_r = mrkrs_femur_r.loc['knee_r_center_in_femur_r']
old_knee_l_in_femur_l = mrkrs_femur_l.loc['knee_l_center_in_femur_l']

old_patellofemoral_r_in_femur_r = mrkrs_femur_r.loc['patella_r_in_femur_r']
old_patellofemoral_l_in_femur_l = mrkrs_femur_l.loc['patella_l_in_femur_l']

# old_walker_knee_r_in_tibia_r = mrkrs_tibia_r.loc['walker_knee_r_in_tibia_r']
# old_walker_knee_l_in_tibia_l = mrkrs_tibia_l.loc['walker_knee_l_in_tibia_l']

old_ankle_r_tibia_r_offset = mrkrs_tibia_r.loc['ankle_r_tibia_r_offset']
old_ankle_l_tibia_l_offset = mrkrs_tibia_l.loc['ankle_l_tibia_l_offset']

old_torso_origin_in_pelvis = mrkrs_pelvis.loc['torso_origin_in_pelvis']

In [148]:
print('new', list(femur_r_center_in_pelvis), 'old', list(old_femur_r_in_pelvis) )

new [-0.04935395, -0.07197149, 0.08605985] old [-0.051425784270424685, -0.06710438124851603, 0.07060125262515124]


In [149]:
new_skin_mrkrs_femur_r = mrkrs_femur_r - (old_femur_r_in_pelvis - femur_r_center_in_pelvis)
new_skin_mrkrs_femur_l = mrkrs_femur_l - (old_femur_l_in_pelvis - femur_l_center_in_pelvis)
#new_skin_mrkrs_femur_r
#new_skin_mrkrs_femur_l

In [150]:
import plotly.express as px
import plotly.graph_objects as go
fig = px.scatter(width = 800, height = 600)
fig.add_trace(go.Scatter3d(x = mrkrs_femur_r['x'], y = mrkrs_femur_r['y'], z=mrkrs_femur_r['z'], mode = 'markers', name = 'osim skin markers'))
fig.add_trace(go.Scatter3d(x = new_skin_mrkrs_femur_r['x'],  y = new_skin_mrkrs_femur_r['y'], z=new_skin_mrkrs_femur_r['z'], mode = 'markers', name = 'new_skin_mrkrs_femur_r mscl and mrkrs'))
fig.add_trace(go.Scatter3d(x = femur_r_df['x'],  y = femur_r_df['y'], z=femur_r_df['z'], mode = 'markers', marker=dict(size=4, opacity=0.8), name = 'transformed mrkrs'))

fig.update_scenes(aspectmode="data" )
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=2, y=2, z=2)
)
fig.update_layout(scene_camera=camera)
fig.show()

In [151]:
import plotly.express as px
import plotly.graph_objects as go
fig = px.scatter(width = 800, height = 600)
fig.add_trace(go.Scatter3d(x = mrkrs_femur_l['x'], 
                           y = mrkrs_femur_l['y'], 
                           z = mrkrs_femur_l['z'], mode = 'markers', name = 'osim skin markers'))
fig.add_trace(go.Scatter3d(x = new_skin_mrkrs_femur_l['x'],  y = new_skin_mrkrs_femur_l['y'], z=new_skin_mrkrs_femur_l['z'], mode = 'markers', name = 'new_skin_mrkrs_femur_r mscl and mrkrs'))
fig.add_trace(go.Scatter3d(x = femur_l_df['x'],  y = femur_l_df['y'], z=femur_l_df['z'], mode = 'markers', marker=dict(size=4, opacity=0.8), name = 'transformed mrkrs'))

fig.update_scenes(aspectmode="data" )
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=2, y=2, z=2)
)
fig.update_layout(scene_camera=camera)
fig.show()

In [152]:
new_skin_mrkrs_tibia_r = mrkrs_tibia_r - (old_knee_r_in_femur_r - knee_r_center_in_femur)
new_skin_mrkrs_tibia_l = mrkrs_tibia_l - (old_knee_r_in_femur_r - knee_r_center_in_femur)

In [153]:
import plotly.express as px
import plotly.graph_objects as go
fig = px.scatter(width = 800, height = 600)
fig.add_trace(go.Scatter3d(x = mrkrs_tibia_l['x'], 
                           y = mrkrs_tibia_l['y'], 
                           z = mrkrs_tibia_l['z'], mode = 'markers', name = 'osim skin markers'))
fig.add_trace(go.Scatter3d(x = new_skin_mrkrs_tibia_l['x'],  y = new_skin_mrkrs_tibia_l['y'], z=new_skin_mrkrs_tibia_l['z'], mode = 'markers', name = 'new_skin_mrkrs_femur_r mscl and mrkrs'))
fig.add_trace(go.Scatter3d(x = tibia_l_df['x'],  y = tibia_l_df['y'], z=tibia_l_df['z'], mode = 'markers', marker=dict(size=4, opacity=0.8), name = 'transformed mrkrs'))

fig.update_scenes(aspectmode="data" )
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=2, y=2, z=2)
)
fig.update_layout(scene_camera=camera)
fig.show()

In [154]:
print(list(talus_r_center_in_tibia))
print(list(old_ankle_r_tibia_r_offset))

[0.00236077, -0.38991173, 0.00219256]
[-0.009759338856214965, -0.39037355424859804, -5.993132995256607e-18]


In [155]:
# new_skin_mrkrs_foot_r = mrkrs_foot_r - (old_ankle_r_tibia_r_offset - talus_r_center_in_tibia)
# new_skin_mrkrs_foot_l = mrkrs_foot_l - (old_ankle_l_tibia_l_offset - talus_l_center_in_tibia)

In [156]:
new_skin_mrkrs_torso = mrkrs_torso - (old_torso_origin_in_pelvis - torso_origin_in_pelvis)

In [157]:
new_skin_mrkrs_pelvis = mrkrs_pelvis - (old_torso_origin_in_pelvis - torso_origin_in_pelvis)

In [158]:
np.linalg.norm(old_torso_origin_in_pelvis - torso_origin_in_pelvis)

0.015990214728619816

In [159]:
import plotly.express as px
import plotly.graph_objects as go
fig = px.scatter(width = 800, height = 600)
fig.add_trace(go.Scatter3d(x = pelvis_df['x'], y = pelvis_df['y'], z=pelvis_df['z'], mode = 'markers', name = 'pelvis_df'))
fig.add_trace(go.Scatter3d(x = mrkrs_pelvis['x'],  
                           y = mrkrs_pelvis['y'], 
                           z = mrkrs_pelvis['z'], mode = 'markers', name = 'osim skin mrkrs'))
fig.add_trace(go.Scatter3d(x = new_skin_mrkrs_pelvis['x'], y = new_skin_mrkrs_pelvis['y'], z=new_skin_mrkrs_pelvis['z'], mode = 'markers', name = 'new_skin_mrkrs_pelvis'))
fig.update_scenes(aspectmode="data" )
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=2, y=2, z=2)
)
fig.update_layout(scene_camera=camera)
fig.show()

## Parse TPS updated model

In [160]:
tree=ET.parse("p03_tps_transformed.osim")
root = tree.getroot()

In [161]:
new_skin_mrkrs_df = pd.concat([new_skin_mrkrs_femur_r, new_skin_mrkrs_femur_l, new_skin_mrkrs_tibia_r, new_skin_mrkrs_tibia_l]) #new_skin_mrkrs_torso,  new_skin_mrkrs_pelvis, new_skin_mrkrs_foot_r, new_skin_mrkrs_foot_l

In [162]:
joint_centers_in_parent = ['torso_origin_in_pelvis','femur_l_center_in_pelvis', 'femur_r_center_in_pelvis',  'knee_l_center_in_femur_l', 'patella_l_in_femur_l', 'knee_r_center_in_femur_r', 'patella_r_in_femur_r', 'ankle_l_tibia_l_offset', 'ankle_r_tibia_r_offset'] 

In [163]:
markers_to_update = list(set(new_skin_mrkrs_df.index) - set(joint_centers_in_parent))

In [164]:
new_skin_mrkrs_df.loc[markers_to_update]

,x,y,z
RTH2,0.001679,-0.294867,0.08664
LTB2,-0.017987,-0.216999,-0.067106
RKNE,0.001534,-0.371979,0.071296
RMKNE,-0.010214,-0.375041,-0.041799
RANK,-0.016438,-0.375831,0.041953
LTB3,0.002007,-0.170793,-0.065186
RTB2,-0.033774,-0.197574,0.076463
LANK,-0.008915,-0.365213,-0.037512
LMMA,-0.01152,-0.350122,0.041297
LKNE,-0.012206,-0.372101,-0.082428


In [165]:
for marker in root.iter('Marker'):
    name = marker.attrib['name']
    # print(name)
    if name in markers_to_update:
        old_text = marker.find('location').text
        new_text = f"{new_skin_mrkrs_df.loc[name, 'x']} {new_skin_mrkrs_df.loc[name, 'y']} {new_skin_mrkrs_df.loc[name, 'z']}"
        marker.find('location').text = new_text

In [166]:
tree.write('p03_tps_transformed_skin_updated.osim')

______________________________________________

### Import  skin markers static TRC

# import .trc file with experimental static marker positions
path_to_static = 'Session2/Static03/task.trc'
static_df = pd.read_csv(path_to_static, delimiter='\t', skiprows=3, header=[0,1], index_col=0)
static_df=static_df.drop('Time', axis = 1)

# adjust marker names
old_lst = list(static_df.columns)
new_lst = []
for i, value in enumerate(old_lst):
    if old_lst[i][0][:2] != 'Un':
        remember = value[0]
        new_lst.append(value)
    else:
        new_lst.append((remember, old_lst[i][1]))
static_df.columns = new_lst

# calculate a series of mean values for each makrer's coordinate
static_means = static_df.mean()[:-1]/1000

# create a dataframe from markers
key = ""
mean_dict = {}
for i, entry in enumerate(static_means.index):
    if entry[0] != key:
        key = entry[0]
        mean_dict[key] = [static_means[i]]
    else:
        mean_dict[key].append(static_means[i])
mean_skin_markers_df = (pd.DataFrame.from_dict(mean_dict)).T
mean_skin_markers_df = mean_skin_markers_df.rename(columns={0:'x', 1:'y', 2:'z'})
